In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [2]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [4]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [5]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [15]:
n,*_ = x_train.shape
c = y_train.max()+1
# number of hidden layers
nh = 32
n,c

(50000, tensor(10))

In [7]:
l1 = nn.Conv2d(1, nh, 5)

In [8]:
x = x_valid[:100]

In [9]:
x.shape

torch.Size([100, 1, 28, 28])

In [10]:
def stats(x): return x.mean(),x.std()

In [11]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [12]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0046, grad_fn=<MeanBackward0>),
  tensor(0.1171, grad_fn=<StdBackward0>)),
 (tensor(-0.0100, grad_fn=<MeanBackward0>),
  tensor(0.1254, grad_fn=<StdBackward0>)))

In [13]:
t = l1(x)

In [14]:
# when not use the kaiming init
stats(t)

(tensor(0.0017, grad_fn=<MeanBackward0>),
 tensor(0.6306, grad_fn=<StdBackward0>))

In [ ]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0267, grad_fn=<MeanBackward1>),
 tensor(1.1067, grad_fn=<StdBackward0>))

In [17]:
import torch.nn.functional as F

In [18]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [19]:
# l1 is the first conv layer 
init.kaiming_normal_(l1.weight, a=0)
# f1 means the activation
stats(f1(x))

(tensor(0.6043, grad_fn=<MeanBackward0>),
 tensor(1.0692, grad_fn=<StdBackward0>))

In [20]:
# l here means the layer
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2064, grad_fn=<MeanBackward0>),
 tensor(0.3065, grad_fn=<StdBackward0>))

> -  When we come here, we found that result of the output of stats(t) is not like the normal kaming result,but what lead to this?
- jeremy want to write his own kaiming function

In [21]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [25]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [22]:
nh

32

In [23]:
# nf means the number of the filters which is equals to the output channels
# ni means number of input
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [26]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [28]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [30]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))
# the last one is the pytorch inner kaiming-init

In [31]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [ ]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5788)

In [32]:
1/math.sqrt(3.)

0.5773502691896258

In [33]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [34]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.6295, grad_fn=<MeanBackward0>),
 tensor(1.1586, grad_fn=<StdBackward0>))

In [35]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2148, grad_fn=<MeanBackward0>),
 tensor(0.3190, grad_fn=<StdBackward0>))

In [36]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [37]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [38]:
y = y_valid[:100].float()

In [39]:
t = m(x)
stats(t)

(tensor(-0.0238, grad_fn=<MeanBackward0>),
 tensor(0.0083, grad_fn=<StdBackward0>))

> Jeremy found that after the conv layers the stats decay so quickly, data stats comes from 1 to 0.4 afer the init and 0.00083 at last.

In [ ]:
l = mse(t,y)
l.backward()

In [ ]:
stats(m[0].weight.grad)

(tensor(0.0054), tensor(0.0333))

In [40]:
init.kaiming_uniform_??

Jeremy make an improvement. He made each layer going through the kaiming_uniform

In [ ]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [ ]:
t = m(x)
stats(t)

(tensor(-0.0352, grad_fn=<MeanBackward1>),
 tensor(0.4043, grad_fn=<StdBackward0>))

In [ ]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0093), tensor(0.4231))

## Export

In [ ]:
!./notebook2script.py 02a_why_sqrt5.ipynb